# Connect to workspace

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

print(azureml.core.VERSION)

# Create compute cluster

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

compute_name = "cpu-cluster"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Compute target exists, will reuse it.')
else:
    print('Creating a new compute target...')
    config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_DS3_V2',
                                                   min_nodes = 0, 
                                                   max_nodes = 1)
    compute_target = ComputeTarget.create(ws, compute_name, config)
    compute_target.wait_for_completion(show_output=True)
    print(compute_target.get_status().serialize())

# Generate experiment and run it in Azure Machine Learning

Create experiment:

In [ ]:
from azureml.core import Experiment

experiment_name = 'german_credit_data_remote'
experiment = Experiment(ws, experiment_name)

Define runtime environmnent:

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment("train-env")
train_env.python.conda_dependencies = CondaDependencies("../azureml/conda.yml")

docker_config = DockerConfiguration(use_docker=True)

Load our dataset and create a script run:

In [ ]:
from azureml.core import ScriptRunConfig

dataset = Dataset.get_by_name(ws, name='german_credit_file')

config = ScriptRunConfig(source_directory='../', 
                         script='train.py',
                         arguments=['--data_path', dataset.as_download()],
                         compute_target=compute_target, 
                         environment=train_env,
                         docker_runtime_config=docker_config)

run = experiment.submit(config=config)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.get_metrics()

## Register model to workspace

In [ ]:
model = run.register_model(model_name='german-credit-basic-model',
                           model_path='outputs/model.pkl',
                           datasets=[['training-dataset', dataset]],
                           tags={'source': 'amlcompute_training_demo'})